### 라이브러리 Imports

In [3]:
import torch
import torch.optim as optim
# For reproducibility
torch.manual_seed(1)

In [4]:
import numpy as np 
import pandas as pd
train_data=pd.read_csv('train_data.csv')
train_data=train_data.drop('ID',axis=1)
train_data

,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize,price
0,2,Focus,2019,1,3764,4,145,34.5,2.3,26299
1,1,I10,2015,1,27056,4,20,60.1,1.0,5495
2,3,A3,2019,3,3375,4,145,34.5,2.0,32400
3,3,A6,2019,0,5743,0,145,47.9,2.0,29990
4,2,Fiesta,2015,1,32782,4,0,65.7,1.0,7250
...,...,...,...,...,...,...,...,...,...,...
17080,0,1 Series,2018,0,4666,4,145,32.5,3.0,23499
17081,3,A4,2020,3,22601,4,145,39.2,2.0,32481
17082,4,Yaris,2018,1,20312,4,145,47.1,1.5,10950
17083,2,Focus,2018,1,18695,4,145,58.9,1.0,13295


In [5]:
CarModel=train_data.groupby('model')
CarModel.groups.keys()

dict_keys([' 1 Series', ' 2 Series', ' 3 Series', ' 4 Series', ' 5 Series', ' 6 Series', ' 7 Series', ' 8 Series', ' A1', ' A3', ' A4', ' A5', ' A6', ' A7', ' A8', ' Auris', ' Avensis', ' Aygo', ' B-MAX', ' C-HR', ' C-MAX', ' Camry', ' Corolla', ' EcoSport', ' Edge', ' Fiesta', ' Focus', ' Fusion', ' GT86', ' Galaxy', ' Getz', ' Grand C-MAX', ' Grand Tourneo Connect', ' Hilux', ' I10', ' I20', ' I30', ' I40', ' I800', ' IQ', ' IX20', ' IX35', ' Ioniq', ' KA', ' Ka+', ' Kona', ' Kuga', ' Land Cruiser', ' M2', ' M3', ' M4', ' M5', ' M6', ' Mondeo', ' Mustang', ' PROACE VERSO', ' Prius', ' Puma', ' Q2', ' Q3', ' Q5', ' Q7', ' Q8', ' R8', ' RAV4', ' RS3', ' RS4', ' RS5', ' RS6', ' S-MAX', ' S3', ' S4', ' S5', ' S8', ' SQ5', ' SQ7', ' Santa Fe', ' Supra', ' TT', ' Tourneo Connect', ' Tourneo Custom', ' Tucson', ' Urban Cruiser', ' Veloster', ' Verso', ' Verso-S', ' X1', ' X2', ' X3', ' X4', ' X5', ' X6', ' X7', ' Yaris', ' Z3', ' Z4', ' i3', ' i8'])

In [6]:
num=0
for i in CarModel.groups.keys():
    train_data.loc[train_data['model']==i,'model']=num
    num+=1

In [7]:
train_data

,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize,price
0,2,26,2019,1,3764,4,145,34.5,2.3,26299
1,1,34,2015,1,27056,4,20,60.1,1.0,5495
2,3,9,2019,3,3375,4,145,34.5,2.0,32400
3,3,12,2019,0,5743,0,145,47.9,2.0,29990
4,2,25,2015,1,32782,4,0,65.7,1.0,7250
...,...,...,...,...,...,...,...,...,...,...
17080,0,0,2018,0,4666,4,145,32.5,3.0,23499
17081,3,10,2020,3,22601,4,145,39.2,2.0,32481
17082,4,93,2018,1,20312,4,145,47.1,1.5,10950
17083,2,26,2018,1,18695,4,145,58.9,1.0,13295


In [8]:
x_train=train_data.copy()
x_train=x_train.drop('price',axis=1)
x_train = torch.FloatTensor(x_train.values.astype(float))


y_train=train_data['price'].copy()
y_train = torch.FloatTensor(y_train.values)
y_train=y_train.reshape(17085,1)

In [9]:
x_train

tensor([[2.0000e+00, 2.6000e+01, 2.0190e+03,  ..., 1.4500e+02, 3.4500e+01,
         2.3000e+00],
        [1.0000e+00, 3.4000e+01, 2.0150e+03,  ..., 2.0000e+01, 6.0100e+01,
         1.0000e+00],
        [3.0000e+00, 9.0000e+00, 2.0190e+03,  ..., 1.4500e+02, 3.4500e+01,
         2.0000e+00],
        ...,
        [4.0000e+00, 9.3000e+01, 2.0180e+03,  ..., 1.4500e+02, 4.7100e+01,
         1.5000e+00],
        [2.0000e+00, 2.6000e+01, 2.0180e+03,  ..., 1.4500e+02, 5.8900e+01,
         1.0000e+00],
        [2.0000e+00, 2.5000e+01, 2.0140e+03,  ..., 0.0000e+00, 7.6400e+01,
         1.5000e+00]])

In [10]:
y_train

tensor([[26299.],
        [ 5495.],
        [32400.],
        ...,
        [10950.],
        [13295.],
        [ 7299.]])

In [11]:
# 모델 초기화
W = torch.zeros((9, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 1000000
for epoch in range(nb_epochs + 1):
    
    hypothesis = x_train.matmul(W) + b

    cost = torch.mean(abs(hypothesis - y_train))

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 997501/1000000 hypothesis: tensor([18952.5078, 11523.8535, 19024.7148,  ..., 16731.2500, 16847.4941,
         9384.8594]) Cost: 5915.596680
Epoch 997502/1000000 hypothesis: tensor([18947.3164, 11487.2305, 19020.0469,  ..., 16703.7266, 16822.1523,
         9333.7217]) Cost: 5915.599609
Epoch 997503/1000000 hypothesis: tensor([18952.5508, 11524.1455, 19024.7539,  ..., 16731.4707, 16847.6973,
         9385.2666]) Cost: 5915.598633
Epoch 997504/1000000 hypothesis: tensor([18947.2930, 11487.0723, 19020.0254,  ..., 16703.6074, 16822.0430,
         9333.5020]) Cost: 5915.599609
Epoch 997505/1000000 hypothesis: tensor([18952.5293, 11523.9863, 19024.7344,  ..., 16731.3516, 16847.5879,
         9385.0449]) Cost: 5915.597656
Epoch 997506/1000000 hypothesis: tensor([18947.2715, 11486.9150, 19020.0059,  ..., 16703.4883, 16821.9355,
         9333.2812]) Cost: 5915.600586
Epoch 997507/1000000 hypothesis: tensor([18952.5078, 11523.8291, 19024.7148,  ..., 16731

In [12]:
print(W)
print(b)

tensor([[ -1.9162],
        [ -1.5891],
        [  7.4205],
        [  1.6737],
        [ -0.1154],
        [ -7.9998],
        [ 34.4579],
        [-15.1627],
        [  3.5269]], requires_grad=True)
tensor([0.0013], requires_grad=True)


In [13]:
import pandas as pd
test_data=pd.read_csv('test_data.csv')
test_data=test_data.drop('ID',axis=1)
test_data

,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize
0,2,Ka+,2017,1,10150,4,145,57.7,1.2
1,2,Kuga,2013,1,50216,0,145,53.3,2.0
2,3,Q5,2016,1,34189,0,200,47.9,2.0
3,1,I10,2020,1,1900,4,145,56.5,1.0
4,0,3 Series,2014,0,79809,0,20,68.9,2.0
...,...,...,...,...,...,...,...,...,...
16826,2,Grand Tourneo Connect,2016,0,29000,0,125,56.6,1.5
16827,4,Yaris,2019,1,2812,4,145,47.9,1.5
16828,2,Kuga,2017,1,46800,0,150,54.3,2.0
16829,3,A7,2017,3,14562,0,145,52.3,3.0


In [14]:
num=0
for i in CarModel.groups.keys():
    test_data.loc[test_data['model']==i,'model']=num
    num+=1

In [15]:
x_test=test_data.copy()
x_test=torch.FloatTensor(x_test.values.astype(float))

x_test

tensor([[2.0000e+00, 4.4000e+01, 2.0170e+03,  ..., 1.4500e+02, 5.7700e+01,
         1.2000e+00],
        [2.0000e+00, 4.6000e+01, 2.0130e+03,  ..., 1.4500e+02, 5.3300e+01,
         2.0000e+00],
        [3.0000e+00, 6.0000e+01, 2.0160e+03,  ..., 2.0000e+02, 4.7900e+01,
         2.0000e+00],
        ...,
        [2.0000e+00, 4.6000e+01, 2.0170e+03,  ..., 1.5000e+02, 5.4300e+01,
         2.0000e+00],
        [3.0000e+00, 1.3000e+01, 2.0170e+03,  ..., 1.4500e+02, 5.2300e+01,
         3.0000e+00],
        [3.0000e+00, 5.9000e+01, 2.0140e+03,  ..., 1.6500e+02, 5.0400e+01,
         2.0000e+00]])

In [16]:
x_test = torch.FloatTensor(x_test)
expect = x_test.matmul(W) + b

expect

tensor([[17817.4453],
        [13262.6250],
        [17087.2539],
        ...,
        [13843.6289],
        [17479.2324],
        [10614.1230]], grad_fn=<AddBackward0>)

In [17]:
id=[]
expected=[]

for i in range(len(expect)):
    id.append(i)
    expected.append(expect[i].item())

In [18]:
submit = pd.DataFrame({'Id':id,
                       'price':expected})
submit.to_csv("submit.csv",index=None)